In [3]:
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

import sys
sys.path.append('../')

from imc.matching_network import Theory

In [15]:
OMEGA = 2 * math.pi * 13.56e6
ZO = complex(50)
ZPL = complex(0.42, 0.1)
    
cl_min=0.02 * 600e-12
cl_max=50 * 1650e-12
step= 0.02 * 50e-12

def gamma(cl):
    zl = complex(1, 0) / complex(0, OMEGA * cl)
    gamma = (zl - complex(ZO)) / (zl + complex(ZO))
    #print("zl=", zl, "  gamma=", gamma)
    return gamma

def gamma_modulus(cl):
    return abs(gamma(cl))

def _ct(cl):
    zl = complex(1, 0) / complex(0, OMEGA * cl)
    t1 = complex(ZO * zl) / complex(ZO + zl)
    t2 = ZPL - complex.conjugate(t1)
    zt = complex.conjugate(t2)
    
    # impedance of an ideal capacitor should have real part equal to zero
    print("zl=", zl, "  zt=", zt)
    # zl is ok, zt is not ok
    
    ct = complex(1, 0) / complex(0, OMEGA * zt)
    xc = ct.imag
    # why is ct complex and possibly with negative real part
    # maybe it is an unacceptable solution, or one of many possible
    #
    # NOTE: from zl and zt printed above: there is a pair of cl and ct
    #       being real capacitors (real=0)
    #       for a given value of zpl
    #
    # this is something to be checked, possibly there shou
    return xc
    
x = np.arange(cl_min, cl_max, step)
y = np.array(list(map(_ct, x)))


# check
def zpl(cl, ct):
    zl = complex(1, 0) / complex(0, OMEGA * cl)
    zt = complex(1, 0) / complex(0, OMEGA * ct)
    zpl2 = complex.conjugate(((ZO - zl) / (ZO + zl)) + zt)
    return zpl2

for i in range(0, x.size):
    zpl2 = zpl(x[i], y[i])
    delta = ZPL - zpl2
    print(i, delta)
# check


X, Y = np.meshgrid(x, y)
x = np.ravel(X)
y = np.ravel(Y)

z = np.array(list(map(gamma_modulus, x)))
Z = z.reshape(X.shape)

#print("===== z =====")
#print(z)
#print("===== Z =====")
#print(Z)

# TODO |Gamma| should depend also on Ct


zl= -978.0908498764464j   zt= (-49.449677867019695+2.4493377160884027j)
zl= -902.8530921936426j   zt= (-49.42712167528479+2.6605333639702287j)
zl= -838.3635856083826j   zt= (-49.40278389530453+2.8714305792008608j)
zl= -782.472679901157j   zt= (-49.37666977454093+3.082006928397251j)
zl= -733.5681374073347j   zt= (-49.34878493488665+3.292240093114289j)
zl= -690.4170705010209j   zt= (-49.31913536965503+3.502107877602186j)
zl= -652.0605665842976j   zt= (-49.28772744037884+3.7115882164722724j)
zl= -617.7415893956503j   zt= (-49.254567873421045+3.9206591822657377j)
zl= -586.8545099258678j   zt= (-49.219663756400685+4.129298992920003j)
zl= -558.9090570722551j   zt= (-49.18302253443767+4.337486019127532j)
zl= -533.5040999326071j   zt= (-49.144652006219935+4.545198791582014j)
zl= -510.3082695007547j   zt= (-49.104560319897004+4.752416008107014j)
zl= -489.0454249382232j   zt= (-49.06275596880378+4.9591165406623015j)
zl= -469.48360794069436j   zt= (-49.019247787018756+5.16527944222325j)
zl= -451.

zl= -0.8473821528061047j   zt= (0.40564299339895205+0.7471388353828757j)
zl= -0.847320978812977j   zt= (0.40564506564363056+0.7470777140723842j)
zl= -0.847259813651726j   zt= (0.4056471374397326+0.7470166015785624j)
zl= -0.8471986573204386j   zt= (0.4056492087873875+0.7469554978995024j)
zl= -0.8471375098172037j   zt= (0.4056512796867249+0.7468944030332987j)
zl= -0.8470763711401097j   zt= (0.4056533501378739+0.7468333169780451j)
zl= -0.8470152412872453j   zt= (0.40565542014096395+0.746772239731836j)
zl= -0.8469541202567007j   zt= (0.40565748969612425+0.7467111712927668j)
zl= -0.846893008046566j   zt= (0.4056595588034841+0.7466501116589332j)
zl= -0.8468319046549321j   zt= (0.4056616274631726+0.7465890608284319j)
zl= -0.8467708100798902j   zt= (0.40566369567531885+0.7465280187993589j)
zl= -0.8467097243195324j   zt= (0.40566576344005195+0.7464669855698123j)
zl= -0.8466486473719512j   zt= (0.40566783075750096+0.7464059611378898j)
zl= -0.8465875792352393j   zt= (0.4056698976277949+0.74634494

zl= -0.5148072370944936j   zt= (0.4147000320254693+0.4147526678571005j)
zl= -0.5147846578297087j   zt= (0.41470049687547067+0.4147300957716562j)
zl= -0.5147620805454742j   zt= (0.41470096166432024+0.41470752566550295j)
zl= -0.5147395052415297j   zt= (0.4147014263920287+0.41468495753838064j)
zl= -0.5147169319176143j   zt= (0.41470189105860683+0.4146623913900287j)
zl= -0.5146943605734677j   zt= (0.4147023556640653+0.4146398272201872j)
zl= -0.5146717912088296j   zt= (0.41470282020841487+0.41461726502859575j)
zl= -0.5146492238234394j   zt= (0.41470328469166623+0.4145947048149944j)
zl= -0.5146266584170369j   zt= (0.41470374911383007+0.4145721465791231j)
zl= -0.5146040949893615j   zt= (0.41470421347491715+0.4145495903207217j)
zl= -0.5145815335401533j   zt= (0.41470467777493814+0.4145270360395301j)
zl= -0.5145589740691521j   zt= (0.4147051420139038+0.4145044837352888j)
zl= -0.5145364165760974j   zt= (0.4147056061918248+0.4144819334077374j)
zl= -0.5145138610607294j   zt= (0.4147060703087118+0.

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# create figure
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(X, Y, Z)

ax.set_xlabel('Cl')
ax.set_ylabel('Ct')
ax.set_zlabel('|Gamma|')

plt.show()
